In [2]:
import torch
import torchvision

In [3]:
# DATASET DOWNLOAD
#dataset = torchvision.datasets.MNIST(root="./data", train=True, download=True, transform=torchvision.transforms.ToTensor())

C:\Users\ashua\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [14]:
train_ds, val_ds = torch.utils.data.random_split(dataset, [50000, 10000])

print(len(train_ds), len(val_ds))


50000 10000


torch.utils.data.dataset.Subset

In [5]:
from torch.utils.data.dataloader import DataLoader

batch_size = 100

train_dl = DataLoader(train_ds, batch_size)
val_dl = DataLoader(val_ds, batch_size)

In [6]:
input_size = 28*28
num_classes = 10

class MnistModel(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = torch.nn.Linear(input_size, num_classes)
        
    def forward(self, image):
        image = image.reshape(-1,784)
        return self.linear(image)
    
    def training_step(self, batch):
        images, labels = batch 
        out = self(images)
        loss = torch.nn.functional.cross_entropy(out, labels) 
        return loss

    def validation_step(self, batch):
        images, labels = batch 
        out = self(images)                    
        loss = torch.nn.functional.cross_entropy(out, labels)   
        acc = accuracy(out, labels)           
        return {'loss': loss, 'accuracy': acc}
    
    def validation_epoch_end(self, outputs):
        batch_losses = [x['loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()
        batch_accuracy = [x['accuracy'] for x in outputs]
        epoch_accuracy = torch.stack(batch_accuracy).mean()
        return {'loss': epoch_loss.item(), 'accuracy': epoch_accuracy.item()}
        
    def epoch_end(self, epoch, result):
        print("Epoch [{}], loss: {:.4f}, accuracy: {:.4f}".format(epoch, result['loss'], result['accuracy']))
        
model = MnistModel()    

print(model.linear.weight.shape,model.linear.bias.shape)
list(model.parameters())

torch.Size([10, 784]) torch.Size([10])


[Parameter containing:
 tensor([[-0.0194,  0.0319,  0.0357,  ...,  0.0130, -0.0153,  0.0258],
         [-0.0199, -0.0257, -0.0349,  ...,  0.0053,  0.0055,  0.0214],
         [-0.0061,  0.0021, -0.0293,  ..., -0.0231,  0.0332,  0.0060],
         ...,
         [-0.0228,  0.0357,  0.0330,  ..., -0.0348,  0.0201, -0.0230],
         [ 0.0176,  0.0059,  0.0241,  ..., -0.0212,  0.0346,  0.0204],
         [-0.0039,  0.0188, -0.0255,  ..., -0.0243, -0.0028, -0.0080]],
        requires_grad=True),
 Parameter containing:
 tensor([-0.0055, -0.0251, -0.0303, -0.0007, -0.0226, -0.0210, -0.0090, -0.0056,
         -0.0182,  0.0085], requires_grad=True)]

In [7]:
for images, label in train_dl:
    print(images.shape)
    outputs = model(images)
    break

torch.Size([100, 1, 28, 28])


In [8]:
def accuracy(outputs, label):
    preds = torch.max(outputs, dim=1)

    return (sum(preds[1] == label)/ len(label))

print(accuracy(outputs,label))

tensor(0.0800)


In [9]:
def fit(epochs, lr, model, train_loader, val_loader, opt_func=torch.optim.SGD):
    optimizer = opt_func(model.parameters(), lr)
    history = [] 
    
    for epoch in range(epochs):
        
        for batch in train_loader:
            loss = model.training_step(batch)
            
            loss.backward()
            
            optimizer.step()
            optimizer.zero_grad()
        
        result = evaluate(model, val_loader)
        model.epoch_end(epoch, result)
        history.append(result)

    return history

In [10]:
def evaluate(model, val_loader):
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

In [11]:
result0 = evaluate(model, val_dl)
print(result0)

{'loss': 2.301473379135132, 'accuracy': 0.10029999911785126}


In [12]:
history1 = fit(3, 0.001, model, train_dl, val_dl)

Epoch [0], loss: 1.8642, accuracy: 0.6398
Epoch [1], loss: 1.5652, accuracy: 0.7357
Epoch [2], loss: 1.3576, accuracy: 0.7731


In [13]:
from PIL import Image
test_dataset = torchvision.datasets.MNIST(root="./data", train=False)

x=4


img, label = test_dataset[x]
img.resize((280,280)).show()

test_dataset = torchvision.datasets.MNIST(root="./data", train=False, transform=torchvision.transforms.ToTensor())

def predict_image(img, model):
    xb = img.unsqueeze(0)
    yb = model(xb)
    print(yb)
    _, preds = torch.max(yb, dim=1)
    print(preds)
    print(_)
    return preds[0].item()

img, label = test_dataset[x]
predict_image(img, model)


tensor([[-0.0595, -1.3121,  0.4483, -0.5267,  0.9997, -0.2762,  0.1878,  0.3818,
          0.1153,  0.6561]], grad_fn=<AddmmBackward>)
tensor([4])
tensor([0.9997], grad_fn=<MaxBackward0>)


4